In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
# Import both models
from models.had_net import HybridAnomalyNet
from models.had_net import DeepOnlyAnomalyNet # (Assuming both are in had_net.py)

# --- Fonction d'évaluation (Utilisant ton code Phase 5) ---
def get_scores(model, loader, center, device, is_hybrid=True):
    model.eval()
    scores, labels = [], []
    with torch.no_grad():
        for images, features, lbl in loader:
            images = images.to(device)
            if is_hybrid:
                outputs = model(images, features.to(device))
            else:
                outputs = model(images)
            dists = torch.sum((outputs - center) ** 2, dim=1)
            scores.extend(dists.cpu().numpy())
            labels.extend(lbl.numpy())
    return scores, labels

# --- Plotting the Comparison ---
# (Assume you have trained both and have scores_hybrid and scores_baseline)

fpr_h, tpr_h, _ = roc_curve(labels_test, scores_hybrid)
fpr_b, tpr_b, _ = roc_curve(labels_test, scores_baseline)

plt.figure(figsize=(8, 6))
plt.plot(fpr_h, tpr_h, color='green', label=f'Proposed Hybrid (AUC = {auc(fpr_h, tpr_h):.4f})')
plt.plot(fpr_b, tpr_b, color='red', linestyle='--', label=f'ResNet18 Baseline (AUC = {auc(fpr_b, tpr_b):.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle=':')
plt.title('Ablation Study: Impact of Texture Features')
plt.legend()
plt.show()